In [ ]:
##SET UP ENVIRONMENT

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import astropy
import os
import eazy
import eazy.hdf5

os.chdir(r"C:\Users\dompo\anaconda3\envs\thesisenv\Lib\site-packages\eazy\data")
cwd = os.getcwd()
print(cwd)

In [ ]:
#files to be used
#extra parameters for fitting of data can also be added here
param_file = 'sub-primer-uds-grizli-v6.0-fix.eazypy.zphot.param'
translate_file = 'sub-primer-uds-grizli-v6.0-fix.eazypy.zphot.translate'
params = {}

In [ ]:
#eazy.photoz.PhotoZ object enables redshift determination from stellar/galactic catalogs
#without the need for full spectroscopic data


#contains JWST+HST photometry
self = eazy.photoz.PhotoZ(param_file=param_file, translate_file=translate_file,
                          zeropoint_file=None, load_prior=True, load_products=True)

In [ ]:
# Turn off error corrections derived above
self.set_sys_err(positive=True)

# Full catalog
sample = np.isfinite(self.ZSPEC)

# fit_parallel renamed to fit_catalog 14 May 2021
self.fit_catalog(self.idx[sample], n_proc=8)

In [ ]:
# quiet numpy/astropy warnings
import warnings
from astropy.utils.exceptions import AstropyWarning

np.seterr(all='ignore')
warnings.simplefilter('ignore', category=AstropyWarning)

# Derived parameters (z params, RF colors, masses, SFR, etc.)
#look to see if i can sample population by mass from self.standard function
#otherwise do it separately
warnings.simplefilter('ignore', category=RuntimeWarning)
zout, hdu = self.standard_output(simple=False, 
                                 rf_pad_width=0.5, rf_max_err=2, 
                                 prior=True, beta_prior=True, 
                                 absmag_filters=[], 
                                 extra_rf_filters=[])

# 'zout' also saved to [MAIN_OUTPUT_FILE].zout.fits

##RUN PROCESS ONLY FOR HUBBLE FILTERS

In [ ]:
#HST photometry only 

translate_file_new = 'sub-primer-uds-grizli-v6.0-fix-hubble.eazypy.zphot.translate'
param_file_new = 'sub-primer-uds-grizli-v6.0-fix.eazypy.zphot.param'
params = {}
params['MAIN_OUTPUT_FILE'] = 'hubble'

In [ ]:
self.hst = eazy.photoz.PhotoZ(param_file=param_file_new, translate_file=translate_file_new,params = params,
                          zeropoint_file=None, load_prior=True, load_products=True)

In [ ]:
# Turn off error corrections derived above
self.hst.set_sys_err(positive=True)

# Full catalog
sample = np.isfinite(self.hst.ZSPEC)

# fit_parallel renamed to fit_catalog 14 May 2021
self.hst.fit_catalog(self.hst.idx[sample], n_proc=8)

In [ ]:
# quiet numpy/astropy warnings
import warnings
from astropy.utils.exceptions import AstropyWarning

np.seterr(all='ignore')
warnings.simplefilter('ignore', category=AstropyWarning)

# Derived parameters (z params, RF colors, masses, SFR, etc.)
#look to see if i can sample population by mass from self.standard function
#otherwise do it separately
warnings.simplefilter('ignore', category=RuntimeWarning)
zout, hdu = self.hst.standard_output(simple=False, 
                                 rf_pad_width=0.5, rf_max_err=2, 
                                 prior=True, beta_prior=True, 
                                 absmag_filters=[], 
                                 extra_rf_filters=[])

# 'zout' also saved to [MAIN_OUTPUT_FILE].zout.fits

In [ ]:
from astropy.table import Table

t = Table.read("sub-primer-uds-grizli-v6.0-fix.eazypy.zout.fits")
#sampling applied, can be changed to other parameters, see catalog header
mask = (t['z_phot']>2) & (t['mass']>1e10)
#masking JWST+HST data
masked_table = t[mask]

tab = Table.read("hubble.zout.fits.zout.fits")
#masking HST data
masked_table_hst = tab[mask]

In [ ]:
#plotting of redshift population histogram

bins = 75

plt.hist(masked_table['z_phot'],bins,color = 'b',alpha=0.2, label='jwst+hst')
plt.hist(masked_table_hst['z_phot'],bins,color = 'g',alpha=0.2, label='hst')
plt.ylim(0,300)
plt.xlim(2,10)
plt.title("JWST + HST vs HST photometric redshift")
plt.ylabel("Number of galaxies")
plt.xlabel("Photometric redshift (z_phot)")
plt.legend(loc='upper right')
plt.show()

In [ ]:
#unmasked version

bins = 75

plt.hist(t['z_phot'],bins, color = 'b', alpha=0.2, label='hst+jwst')
plt.hist(tab['z_phot'],bins,color = 'g', alpha=0.2, label='hst')
plt.ylim(0,10000)
plt.xlim(2,18)
plt.title("Unmasked - JWST + HST vs HST photometric redshift")
plt.ylabel("Number of galaxies")
plt.xlabel("Photometric redshift (z_phot)")
plt.legend(loc='upper right')
plt.show() 

In [ ]:
#unmasked masses plot

plt.plot(np.log10(tab['mass']), np.log10(t['mass']),'ko',alpha=.2,ms=1)
plt.plot([0,18],[0,18],color='red')
plt.xlim(6,13)
plt.ylim(6,13)
plt.title("Unmasked - JWST + HST vs HST masses")
plt.xlabel("HST Solar masses (log10(M))")
plt.ylabel("JWST Solar masses (log10(M))")
plt.show()


In [ ]:
#masked mass comparison plot

plt.plot(np.log10(masked_table_hst['mass']),np.log10(masked_table['mass']),'ko',alpha=.5,ms=1)
plt.plot([10,18],[10,18],color='red')
plt.xlim(10,13)
plt.ylim(10,13)
plt.title("JWST + HST vs HST solar masses")
plt.xlabel("HST Solar masses (log10(M))")
plt.ylabel("JWST+HST Solar masses (log10(M))")
plt.show()

jwst_median_mass = np.median((masked_table['mass']),axis = 0)
hst_median_mass = np.median((masked_table_hst['mass']),axis = 0)
jwst_mean_mass = np.mean((masked_table['mass']))
hst_mean_mass = np.mean((masked_table_hst['mass']))
jwst_std_mass = np.std((masked_table['mass']))
hst_std_mass = np.mean((masked_table_hst['mass']))

#hts_median_mass_log = np.log10(hst_median_mass)

print("Median JWST: ", jwst_median_mass)
print("Median HST: ", hst_median_mass)
print("Mean JWST: ", jwst_mean_mass)
print("Mean HST: ", hst_mean_mass)
print("St dev JWST: ", jwst_std_mass)
print("St dev HST: ", hst_std_mass)

In [ ]:
#plot of mass difference in terms of HST observed mass (logscale)

delta_mass_log = np.log10(masked_table['mass'])-np.log10(masked_table_hst['mass'])
delta_mass = (masked_table['mass'])-(masked_table_hst['mass'])
plt.plot(np.log10(masked_table_hst['mass']),delta_mass_log,'ko',alpha=.5,ms=1)

plt.xlim(10,13)
plt.plot([0,16],[0,0],color='red')
plt.title("Solar Mass difference between methods")
plt.xlabel("HST Solar masses (log10(M))")
plt.ylabel("Difference Solar masses (log10(M))")
plt.show()

jwst_median_massdiff = np.median(delta_mass,axis = 0)
jwst_mean_massdiff = np.mean(delta_mass)
jwst_std_massdiff = np.std(delta_mass)

print("Median massdiff: ", jwst_median_massdiff)
print("Mean massdiff: ", jwst_mean_massdiff)
print("St dev massdiff: ", jwst_std_massdiff)

In [ ]:
#photometric redshift JWST+HSTvs photometric redshift HST

plt.plot((masked_table_hst['z_phot']),(masked_table['z_phot']),'ko',alpha=.5,ms=1)
plt.xlim(2,10)
plt.ylim(2,10)
plt.plot([2,16],[2,16],color='red')
plt.title("Photometric redshift JWST+HST vs HST")
plt.xlabel("HST photometric redshift")
plt.ylabel("JWST+HST photometric redshift")
plt.show()

jwst_median_z = np.median((masked_table['z_phot']),axis = 0)
hst_median_z = np.median((masked_table_hst['z_phot']),axis = 0)
jwst_mean_z = np.mean((masked_table['z_phot']))
hst_mean_z = np.mean((masked_table_hst['z_phot']))
jwst_std_z = np.std((masked_table['z_phot']))
hst_std_z = np.mean((masked_table_hst['z_phot']))

print("Median JWST z: ", jwst_median_z)
print("Median HST z: ", hst_median_z)
print("Mean JWST z: ", jwst_mean_z)
print("Mean HST z: ", hst_mean_z)
print("St dev JWST z: ", jwst_std_z)
print("St dev HST z: ", hst_std_z)

In [ ]:
#delta z / (1+z) vs z shows more intuitively how the difference in redshifts scales 

delta_z = masked_table['z_phot'] - masked_table_hst['z_phot']
plt.plot((masked_table_hst['z_phot']),delta_z/(1+masked_table_hst['z_phot']),'ko',alpha=.5,ms=1)
plt.xlim(2,10) 
plt.ylim(-1,4)
plt.plot([0,13],[0,0],color='red')
plt.title("Photometric redshift difference")
plt.xlabel("HST photometric redshift")
plt.ylabel("Delta z / (1+z)")
plt.show()

jwst_median_z = np.median(delta_z,axis = 0)
jwst_mean_z = np.mean(delta_z)
jwst_std_z = np.std(delta_z)

print("Median zdiff: ", jwst_median_z)
print("Mean zdiff: ", jwst_mean_z)
print("St dev zdiff: ", jwst_std_z)

In [ ]:
# delta z/z vs delta solar mass

delta_z = masked_table['z_phot'] - masked_table_hst['z_phot']
plt.plot(delta_mass_log,delta_z/(masked_table_hst['z_phot']),'ko',alpha=.5,ms=1)
plt.xlim(-4,4)
plt.ylim(-1,1.5)
plt.plot([-4,4],[0,0],color='red')
plt.title("Photometric redshift difference vs solar mass difference")
plt.xlabel("Difference in solar mass (log10(M))")
plt.ylabel("Delta z/z")
plt.show()

In [ ]:
# stellar mass vs z_phot JWST and HST same graph

delta_z = masked_table['z_phot'] - masked_table_hst['z_phot']
plt.plot(masked_table['z_phot'],np.log10(masked_table['mass']),'ko',alpha=.5,ms=1, label = 'JWST+HST')
plt.plot(masked_table_hst['z_phot'],np.log10(masked_table_hst['mass']),'ro',alpha=.5,ms=1, label = 'HST')
plt.ylim(10,13)
plt.xlim(2,10)

plt.title("JWST+HST and HST Solar mass vs photometric redshift")
plt.ylabel("Solar mass (log10(M))")
plt.xlabel("Redshift")
plt.legend(loc="upper right")
plt.show()

## JWST+HST SED

In [ ]:
# Show brightest objects with z_spec > 1
params['PRIOR_ABZP'] = 23.9

ifilter = self.flux_columns[np.argmin((self.lc - 8140)**2)]

imag = params['PRIOR_ABZP'] - 2.5*np.log10(self.cat[ifilter])
#sel = (self.ZSPEC == -1.0) #all sources
sel = (t['id'] == 124) 
#sample: (zout['z_phot'] > 2) & (zout['z_phot'] < 3) & (zout['mass'] > 10e10) 3)

so = np.argsort(imag[sel])
ids = self.OBJID[sel][so]


fig1, data1 = self.show_fit(ids[0], xlim=[0.2, 9], show_components=True,
                              logpz=True, zr=[2,5])

## JWST+HST and HST SED with one id input

In [ ]:
# Show brightest objects with z_spec > 1
params['PRIOR_ABZP'] = 23.9

ifilter_hst = self.hst.flux_columns[np.argmin((self.hst.lc - 8140)**2)]

imag_hst = params['PRIOR_ABZP'] - 2.5*np.log10(self.hst.cat[ifilter_hst])
#sel = (self.ZSPEC == -1.0) #all sources
sel = (t['id'] == 22367)
sel_hst = (t['id'] == 22367)
#sample: (tab['z_phot'] > 2) &  (tab['mass'] > 10e10)  

so_hst = np.argsort(imag_hst[sel_hst])
ids_hst = self.hst.OBJID[sel_hst][so_hst]

fig, data = self.hst.show_fit(ids_hst[0], xlim=[0.2, 12], show_components=True,
                              logpz=False, zr=[2,5])
fig1, data1 = self.show_fit(ids[0], xlim=[0.2, 12], show_components=True,
                              logpz=False, zr=[2,5])
    

In [ ]:
###Uncomment the input command for ease of use
#intended to be used with a provided id
#replace the sampling with the commented ide
#also includes the saving features and tuple for ID and chi2 value of fit
params['PRIOR_ABZP'] = 23.9

ifilter = self.flux_columns[np.argmin((self.lc - 8140)**2)]

imag = params['PRIOR_ABZP'] - 2.5*np.log10(self.cat[ifilter])

#ide = int(input("Input galaxy id: "))
#(t['id'] == ide)
sel = (tab['z_phot'] > 2) &  (tab['mass'] > 10e10)

so = np.argsort(imag[sel])
ids = self.OBJID[sel][so]



ifilter_hst = self.hst.flux_columns[np.argmin((self.hst.lc - 8140)**2)]

imag_hst = params['PRIOR_ABZP'] - 2.5*np.log10(self.hst.cat[ifilter_hst])

sel_hst = (tab['z_phot'] > 2) &  (tab['mass'] > 10e10)
#can use similar sampling as previously
so_hst = np.argsort(imag_hst[sel_hst])
ids_hst = self.hst.OBJID[sel_hst][so_hst]

extend_list = []
    
for i in range(1764):
    data = self.hst.show_fit(ids_hst[i], get_spec = True, xlim=[0.2, 12], show_components=True,
                              logpz=True, zr=[3,6])
    plt.savefig(str(ide) + ' - hst.png', bbox_inches='tight')


    data1 = self.show_fit(ids[i], get_spec = True,xlim=[0.2, 12], show_components=True,
                              logpz=True, zr=[3,5])
    plt.savefig(str(ide) + ' - jwst.png', bbox_inches='tight')
    extend_list.extend((data1['id'], data1['chi2'])) # extending tuple elements

In [ ]:
#routine for separating values from the ID and chi2 tuple, plot and printing of the values

idlist = []
l_id = []
l_chi2 = []
for i in range(2*1764):
    if(i%2 == 0):
        l_id.append(extend_list[i])
    else:
        l_chi2.append(extend_list[i])

chi2_list_min = sorted(l_chi2)[:30]
#print(chi2_list_min)
a = np.array(l_id)
b = np.array(l_chi2)

for i in chi2_list_min:
    for j in range(1764):
        if i == b[j]:
            print("Chi: ",a[j],"ID: ",print(i))

plt.plot((masked_table['z_phot']), l_chi2, 'ko',ms = 0.4, alpha = 1)

In [ ]:
#plot for the difference of SEDs in the same figure (flux vs wavelength)

plt.plot(np.log10(data['templz']),(data['templf']), label = "JWST")
plt.plot(np.log10(data1['templz']),(data1['templf']), label = "HST")
plt.ylim(0,1000)
plt.xlim(2.5,8)
plt.xlabel("Wavelength")
plt.ylabel("Flux")
plt.title("Source id: " + str(ide))
plt.legend(loc = "upper right")
plt.savefig(str(ide) + ' - both.png', bbox_inches='tight')
plt.show()

diff_flux = (data1['templz']) - (data['templz'])
diff_wave = np.log10(data1['templf'] - np.log10(data['templf']))

plt.show()

In [ ]:
#SFR plot masked and unmasked

plt.plot(np.log10(t['sfr']),np.log10(tab['sfr']),'ko',alpha=.2,ms=1)
plt.plot([0,8],[0,8],color='red')
plt.xlim(0,8)
plt.ylim(0,8)
plt.title("Unmasked - JWST + JST vs HST SFR")
plt.xlabel("SFR (log10(Solar masses/year))")
plt.ylabel("SFR (log10(Solar masses/year))")
plt.show()

plt.plot(np.log10(masked_table['sfr']),np.log10(masked_table_hst['sfr']),'ko',alpha=.2,ms=1)
plt.plot([0,8],[0,8],color='red')
plt.xlim(0,8)
plt.ylim(0,8)
plt.title("Masked - JWST + JST vs HST SFR")
plt.xlabel("SFR (log10(Solar masses/year))")
plt.ylabel("SFR (log10(Solar masses/year))")
plt.show()

In [ ]:
#HST SEDs in a range

params['PRIOR_ABZP'] = 23.9

ifilter = self.flux_columns[np.argmin((self.lc - 8140)**2)]

imag = params['PRIOR_ABZP'] - 2.5*np.log10(self.cat[ifilter])
#sel = (self.ZSPEC == -1.0) #all sources 
  
#sample, (zout['z_phot'] > 2) & (zout['z_phot'] < 3) & (zout['mass'] > 10e10) 3)
sel = (tab['z_phot'] > 4) &  (tab['mass'] > 10e10) & (tab['z_phot'] < 4.5)
so = np.argsort(imag[sel])
ids = self.OBJID[sel][so]
for i in range(10):
    data1 = self.show_fit(ids[i], xlim=[0.2, 12], show_components=True,
                              logpz=True, zr=[2,8])